### Подключаем библиотеки

In [1]:
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkFiltersSources import vtkSphereSource
from vtkmodules.vtkIOImage import (
    vtkBMPWriter,
    vtkJPEGWriter,
    vtkPNGWriter,
    vtkPNMWriter,
    vtkPostScriptWriter,
    vtkTIFFWriter
)
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkRenderer,
    vtkWindowToImageFilter
)

from vtk import vtkPolyDataReader, vtkTransform, vtkPlane

import os

import cv2
import numpy as np

### Генерируем фронтальную проекцию vtk модели

In [2]:
VTK_INPUT_FILE = os.path.join("assets", "input.vtk")
OUTPUT_FILE = os.path.join('output', 'frontProjection')

In [6]:
def main():
    colors = vtkNamedColors()

    # Set the background color.
    colors.SetColor('BkgColor', [0, 0, 0, 255])

    reader = vtkPolyDataReader()
    reader.SetFileName(VTK_INPUT_FILE)
    reader.Update()

    data = reader.GetOutput()

    # create a rendering window and renderer
    ren = vtkRenderer()
    renWin = vtkRenderWindow()
    renWin.ShowWindowOff()
    renWin.AddRenderer(ren)
    renWin.SetSize(1024, 1024)

    # create a renderwindowinteractor
    # iren = vtkRenderWindowInteractor()
    # iren.SetRenderWindow(renWin)

    # create source
    source = vtkSphereSource()
    source.SetCenter(0, 0, 0)
    source.SetRadius(5.0)

    # mapper
    mapper = vtkPolyDataMapper()
    mapper.SetInputData(data)

    # actor
    actor = vtkActor()
    actor.SetMapper(mapper)

    # color the actor
    actor.GetProperty().SetColor(colors.GetColor3d('Yellow'))

    # assign actor to the renderer
    ren.AddActor(actor)
    ren.SetBackground(colors.GetColor3d('BkgColor'))

    center_x, center_y, center_z = actor.GetCenter()
    w = vtkTransform()
    w.Translate(-center_x, -center_y, -center_z)
    # set up needed angle
    # w.RotateY(180)
    w.RotateX(-90)
    actor.SetUserTransform(w)

    renWin.SetWindowName('ImageWriter')
    renWin.Render()

    # ext = ['', '.png', '.jpg', '.ps', '.tiff', '.bmp', '.pnm']
    ext = ['.png']
    filenames = list(map(lambda x: OUTPUT_FILE + x, ext))
    for f in filenames:
        writeImage(f, renWin, rgba=False)


def writeImage(fileName, renWin, rgba=True):
    # Select the writer to use.
    path, ext = os.path.splitext(fileName)
    ext = ext.lower()
    if not ext:
        ext = '.png'
        fileName = fileName + ext
    if ext == '.bmp':
        writer = vtkBMPWriter()
    elif ext == '.jpg':
        writer = vtkJPEGWriter()
    elif ext == '.pnm':
        writer = vtkPNMWriter()
    elif ext == '.ps':
        if rgba:
            rgba = False
        writer = vtkPostScriptWriter()
    elif ext == '.tiff':
        writer = vtkTIFFWriter()
    else:
        writer = vtkPNGWriter()

    windowto_image_filter = vtkWindowToImageFilter()
    windowto_image_filter.SetInput(renWin)
    windowto_image_filter.SetScale(1)
    if rgba:
        windowto_image_filter.SetInputBufferTypeToRGBA()
    else:
        windowto_image_filter.SetInputBufferTypeToRGB()
        windowto_image_filter.ReadFrontBufferOff()
        windowto_image_filter.Update()

    writer.SetFileName(fileName)
    writer.SetInputConnection(windowto_image_filter.GetOutputPort())
    writer.Write()


main()

### Генерируем контур вдоль носового отверстия

In [23]:
CONTOUR_INPUT = os.path.join('output', 'frontProjection.png')
CONTOUR_OUTPUT = os.path.join('output', 'contouredNose.png')

In [50]:
img = cv2.imread(CONTOUR_INPUT)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

blur = cv2.medianBlur(gray, 27)

cv2.imwrite(CONTOUR_OUTPUT, blur)
ret, thresh = cv2.threshold(blur, 110, 255, cv2.THRESH_OTSU)

canny = cv2.Canny(thresh, 100, 200)

contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

contour_list = []
for contour in contours:
    approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)
    area = cv2.contourArea(contour)
    contour_list.append(contour)

# Biggest contour are outter
contour = sorted(contour_list, key=lambda c: cv2.contourArea(c))[-3]

cv2.drawContours(img, [contour], -1, (0, 255, 0), 2)

cv2.imwrite(CONTOUR_OUTPUT, img)


True